# Quickly build a chain with output parser

In [2]:
from setup import get_openai_model, enable_tracing
enable_tracing("seminar")
MODEL = get_openai_model()

Enable tracing at seminar
OPENAI ready


# Text in text out

In [4]:
question = "The weather is hot"
ans = MODEL(question)
print(ans)



If the weather is hot, it is best to stay hydrated by drinking plenty of water and avoiding strenuous activities during the hottest parts of the day. It is also important to wear light, breathable clothing and to use sunscreen to protect your skin from the sun's harmful rays.


# PromptTemplate

In [10]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

solution_template = """"
You're a sale consultant and you're capable of giving shopping recommendations.
Given a customer condition, you can list options along with a brief uses
 expalaination for each option. You should only list out {n_options} options\
  and must response in {lang}.
The question: {condition}. What should I buy? 
The answer: """

solution_prompt = PromptTemplate(input_variables=['condition','n_options','lang'],\
                                  template=solution_template)

solution_chain = LLMChain(llm=MODEL, prompt=solution_prompt,\
                               output_key='solution_list')

res = solution_chain({"condition":"Ngoài trời nóng quá"\
                              ,"n_options":"2", 'lang':'English'})
print(res)

{'condition': 'Ngoài trời nóng quá', 'n_options': '2', 'lang': 'English', 'solution_list': '\n1. A portable air conditioner: This is a great option if you need to cool down a specific area quickly. \n2. A ceiling fan: This is a great option if you need to circulate air and keep a room cool without using too much energy.'}


In [9]:
res = solution_chain({"condition":"Ngoài trời nóng quá"\
                              ,"n_options":"3", 'lang':'English'})
print(res)

{'condition': 'Ngoài trời nóng quá', 'n_options': '3', 'lang': 'English', 'solution_list': '\n1. A portable air conditioner: This is a great option for cooling down a room quickly and efficiently. \n2. A fan: A fan is a great way to circulate air and keep a room cool. \n3. A cooling pillow: A cooling pillow is a great way to keep your head cool and comfortable during hot days.'}


# Output Parser

In [28]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain import LLMChain

# define the output
response_schemas = [
    ResponseSchema(name="options", description="a js array with {n_options} shopping options to solve the user question"),
    ResponseSchema(name="reason", description="the reason behind your answer"),
    ResponseSchema(name="source", description="the source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# prompt template
template = """Answer the user question as best as possible, and you must speak in {language}.
{format_instructions}
The question: {question} """

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(template=template, input_variables=["question", "language"],\
                        partial_variables={"format_instructions": format_instructions})

# Optional part
input_prompt = prompt.format_prompt(question="The weather outside is too hot",\
                                    language="English")
print(input_prompt.to_string())

# Build chain
solution_chain = LLMChain(llm=MODEL, prompt=prompt, output_key='result')

Answer the user question as best as possible, and you must speak in English.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"options": string  // a js array with {n_options} shopping options to solve the user question
	"reason": string  // the reason behind your answer
	"source": string  // the source used to answer the user's question, should be a website.
}
```
The question: The weather outside is too hot 


In [30]:
res = solution_chain({"question":"The weather outside is too hot"\
                              ,"n_options":"3", 'language':'English'})
print(res)

{'question': 'The weather outside is too hot', 'n_options': '3', 'language': 'English', 'result': '\n\n```json\n{\n\t"options": ["Buy an air conditioner", "Buy a fan", "Buy a portable air cooler"],\n\t"reason": "These options will help to cool down the temperature in your home.",\n\t"source": "https://www.consumerreports.org/cro/air-conditioners/buying-guide/index.htm"\n}\n```'}


Congrats! You've learned how to build a simple chain with output parser